In [1]:
# Test on file 1
import practice2
import matplotlib.pyplot as plt
import pandas as pd
import math
import time

In [2]:
directory = "Practice_02_data/"
list_data = practice2.list_file_data(directory)
filename = str(directory+list_data[0])

In [5]:
pl_2, lt_2 = practice2.text_mining(filename,True)


0.38521599769592285


In [138]:
posting = {}
posting['a'] = [('1', 1), ('2',1)]
posting['b'] = [('2',4)]
posting['c'] = [('1', 1), ('2',1)]
posting['d'] = [('1', 4)]
posting['e'] = [('1', 5)]


In [139]:
stat = {}
stat['dl'] = [('1',11),('2',6)]

df = {}
df['a'] = 10
df['b'] = 25
df['c'] = 10
df['d'] = 24
df['e'] = 250
n = 1000
avdl = 20
k = 1.0
b = 0.5
q = 'a e'

In [190]:
def smart_ltn(posting_list,n, df_t):
    df = 0
    #(1+LOG(tf))*LOG(n/df)
    #tf is the frequence of term on a document
    #df number of document where the term appear
    ltn_value = {} # key doc number, value (terme, ltn)
    for term, value in posting_list.items():
        df = df_t[term] #
        for v in value:
            tf = v[1]
            if tf == 0:
                ltn=0.0
            else:
                ltn = (1+math.log10(tf))*math.log10(n/df)
            ltn_value.setdefault(v[0],[]).append((term,round(ltn,4)))
    return ltn_value

def rsv_score(query,ltn):
    rsv_doc = {}
    for term in query.split():
        for doc, values in ltn.items():
            for v in values:
                if term in v[0]:
                    rsv_doc.setdefault(doc,[]).append(v[1]) 
                else: continue
    rsv = [(sum(ltn_list),doc) for doc,ltn_list in rsv_doc.items()]
    return rsv

def smart_ltc(smart_ltn):
    # ltn = doc:(term, ltn)
    #  (ltn_term/sqrt(sum(each pow(ltn_term,2))on a document))
    ltc_value = {}
    for doc, value in smart_ltn.items():
        pow_ltn_term = [pow(v[1],2) for v in value]
        normalize = math.sqrt(sum(pow_ltn_term))
        for v in value:
            ltc_value.setdefault(doc,[]).append((v[0],round((v[1]/normalize),4)))
    return ltc_value


In [191]:
ltn = smart_ltn(posting, n,df)
print(rsv_score(q,ltn))
ltc = smart_ltc(ltn)
print(rsv_score(q,ltc))


[(3.0229, '1'), (2.0, '2')]
[(0.7609999999999999, '1'), (0.5236, '2')]


In [186]:
def tf_part(posting_list,dl,n):
    #(tf*(k+1))/(k*((1-b)+b*(dl/avdl))+tf)
    #[  1     ]             [   2   ]
    #              [      3         ]
    #            [         4            ]   
    k = 1.0
    b = 0.5
    dl_ = [dl_ for _, dl_ in dl]
    doc_len = 0
    #avdl = (sum(dl_))/n
    avdl = 20
    tf_part_val = {}
    for term, value in posting_list.items():
        for v in value:
            #if not term in v
            tf = v[1]
            doc_len = [doc_l for docno,doc_l in dl if docno==v[0]]
            #doc_len = dl[v[0]]
            bloc_1 = tf*(k+1)
            bloc_2 = doc_len[0]/avdl
            bloc_3 = k * ((1-b) + b * bloc_2)
            bloc_4 = bloc_3+tf       
            tf_part_val.setdefault(v[0],[]).append((term,bloc_1/bloc_4)) 
    return tf_part_val

def idf_part(posting_list,df_,n):
    #log((n-df+0.5)/(df+0.5))
    #    [   1    ] [  2   ]
    idf_part_val ={}
    for term, value in posting_list.items():
        df = df_[term]
        bloc_1 = n-df+0.5
        bloc_2 = df + 0.5
        
        idf_part_val.setdefault(term,[]).append(math.log10(bloc_1/bloc_2))
    return idf_part_val

def bm25(posting_list, stat,df_,n):
    tf_part_ = tf_part(posting_list,stat['dl'],n)

    idf_part_ = idf_part(posting_list,df_,n)
    bm25_val = {}
    # doc 1 : ('a' , tf_part_[a]*idf[a])
    for doc, tf_value in tf_part_.items():
        for tf in tf_value:
            bm25_val.setdefault(doc,[]).append((tf[0], tf[1]*idf_part_[tf[0]][0]))     
    return bm25_val


In [187]:
tf_part(posting,stat['dl'],n)
#idf = idf_part(posting,df,n)
#idf['a']
bm25_ = bm25(posting,stat,df,n)
#rsv_score(q,bm25)

In [188]:
rsv_score(q,bm25_)

[(3.0501574773701985, '1'), (2.393533552490458, '2')]